# New Road Analysis  
We start by building the athlete historical activities. Each activities path is sampled uniformly (by 500m by default).  
For a new activity, we compare each new path segment with the historical ones.

In [3]:
%load_ext autoreload
%autoreload 2

First, we set up the Strava client. It's a bit annoying to get the refresh token that has a short life span by default. 

In [18]:
from wandrer.client import StravaClient

strava_credentials = StravaClient(
    access_token='cd4b34f5a69d66ea7a85bc973e6bb224233c34e5',
    refresh_token='86b84f2a3a9d1eb288b4b12d2990b7ee1aebfcff',
    client_id='8712',
    client_secret='30509b6d35b37543cfe85bee1018db8fee15e22c'
)

strava_credentials.configure_client()
activities = strava_credentials.get_activities()

We can build the historical activities datasets (with `fetch_path_segments`) before indexing it in a `BallTree`.  
By default, we only fetch the last 10 activities (`limit`).

In [6]:
from wandrer.activity import HistoricalActivities

history = HistoricalActivities(strava_client=strava_credentials)
history.fetch_path_segments(limit=10)
history.index()

We can visualize all historical path segments.

In [22]:
import folium

folium_map = folium.Map([48.862, 2.346], zoom_start=11)
markers = [folium.Marker(loc, icon=None, popup="").add_to(folium_map) for loc in [p for segment in history.path_segments for p in segment]]
folium_map

For a new activity, we can compare its path segments with the historical ones as follows. We then display its new segments if any.

In [32]:
from wandrer.activity import Activity

new_activity = Activity(activity_id = activities[-1].id, strava_client=strava_credentials)
new_activity.fetch_path_segments()
new_segments = new_activity.get_new_path_segments(history=history)

folium_map = folium.Map(new_segments[0][0], zoom_start=11)
markers = [folium.Marker(loc, icon=None, popup="").add_to(folium_map) for loc in [p for segment in new_activity.path_segments for p in segment]]

folium_map

Indeed, it's the only time I went to Provins.

In [13]:
new_activity.strava_segments

[{'name': 'commico to lidl',
  'start_latlng': [48.842563, 2.3849],
  'end_latlng': [48.844209, 2.381499]},
 {'name': 'COMICO-ANNEAU VINCENNES',
  'start_latlng': [48.833912, 2.444888],
  'end_latlng': [48.844112, 2.381777]},
 {'name': 'Daumesnil : portion rambouillet-bizot',
  'start_latlng': [48.837396, 2.401411],
  'end_latlng': [48.84404, 2.381955]},
 {'name': 'Montée Av Daumesnil - 900m',
  'start_latlng': [48.840064, 2.393944],
  'end_latlng': [48.843364, 2.383412]},
 {'name': 'Av. Daumesnil',
  'start_latlng': [48.840015, 2.394157],
  'end_latlng': [48.841453, 2.387454]},
 {'name': 'Daumesnil / Porte Dorée',
  'start_latlng': [48.835838, 2.405478],
  'end_latlng': [48.839137, 2.396071]},
 {'name': 'Avenue Daumesnil to Porte Dorée',
  'start_latlng': [48.835709, 2.40598],
  'end_latlng': [48.839222, 2.396562]},
 {'name': 'Av. Daumesnil vers Vincennes (full)',
  'start_latlng': [48.840467, 2.434198],
  'end_latlng': [48.839151, 2.396793]},
 {'name': 'porte Dorée > château de Vince